# Gaussian Bayesian Classifier

### Mastery Checkpoint 1  
CSC 466, Winter 2022  
Samay Nathani

In [6]:
from pathlib import Path
home = str(Path.home())

In [7]:
%load_ext autoreload
%autoreload 2

import Lab2_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Import our datasets

In [9]:
import pandas as pd
import numpy as np
titanic_df = pd.read_csv(
    f"{home}/csc-466-student/data/titanic.csv"
)

Defining the Gaussian Bayesian Classifier

In [14]:
class GaussianBayes:
    
    def compute_priors(y):
        value_counts = y.value_counts().sort_index()
        total_values = len(y)
        indexes = [y.name+"="+str(v) for v in value_counts.index.tolist()]
        priors = dict(zip(indexes, value_counts / total_values))
        return priors
    
    
    # Since we are assuming a normal distribution, this is where we use the mean, std, and variance to calculate the class conditional probability. 
    def specific_class_conditional(x,xv,y,yv):
        # todo: replace binning with normal distribution calculations
        likelihoods = {}
        priors = {}
        
        for ux in x.unique():
            binned, bins = pd.cut(y.loc[x==ux], 5, retbins=True)
            binned_counts = binned.value_counts()
            bin_value = pd.cut([yv], bins)[0]
            
            if bin_value in binned_counts.index:
                count = binned_counts.loc[bin_value]
            else:
                count = 0
            
            likelihoods[ux] = count / sum(binned_counts)
            #priors[ux] = sum(binned_counts) / len(x)

        denom = 0
        
        #for k in priors.keys():
        #    denom += (priors[k]*likelihoods[k])

        #classcond = likelihoods[xv]*priors[xv] / denom
        #return classcond


    def class_conditional(X,y):
        probs = {}
        
        for eachy in y.unique():
            for col in X.columns:
                for eachx in X[col].unique():
                    probs[col + "=" + str(eachx) + "|" + y.name + "=" + str(eachy)] = specific_class_conditional(X[col], eachx, y, eachy)
        return probs

    def posteriors(probs,priors,x):
        post_probs = {}
        denom = 0
        for k in priors.keys():
            numerator = 1
            postkey = ""
            
            for idx in x.index:
                postkey += idx + "=" + str(x[idx]) + ","
                probkey = idx + "=" + str(x[idx]) + "|" + k
                
                if probkey not in probs:
                    numerator *= 0
                else:
                    numerator *= probs[probkey]
            
            numerator*=priors[k]
            post_probs[k + "|" + postkey[:-1]] = numerator
            denom+=numerator
            
        for k, v in post_probs.items():
            if denom != 0:
                post_probs[k] = v / denom
            else:
                post_probs[k] = 1 / len(list(priors.keys()))
        
        return post_probs
